# 🚀 머신러닝 실습 : 고객 구매 데이터로 성별 예측 모델링 (분류 문제)

* 주어진 데이터는 백화점 고객의 1년 간 구매 데이터입니다.
* 고객 3,500명에 대한 학습용 데이터(y.csv, X.csv)를 이용하여 성별예측 모형을 만들어보세요.
* 모델의 성능은 자유롭게 측정해봅니다!

## [실습 프로세스]
1. 데이터 불러오기  
2. 데이터 탐색
3. 데이터 전처리  
4. 학습/테스트 데이터 분리  
5. 모델 선택 및 학습  
6. 예측 및 평가  


<br/>

---

<br/>
<br/>

# 0. 라이브러리 불러오기

* 라이브러리를 가져와서 과정을 준비합니다

In [99]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

<br/>

---

<br/>
<br/>

# 1. 데이터 불러오기
* 데이터를 가져와서 과정을 준비합시다.
- 인코딩 방식은 'euc-kr' 을 활용하세요.
- 데이터 출처 : 한국데이터산업진흥원 빅데이터분석기사 실기 공개 예시 문항

- 독립 변수 데이터셋 : ./data/X.csv
- 종속 변수 데이터셋 : ./data/y.csv


데이터 파일을 불러옵니다. 보통 CSV 파일을 pandas로 읽어옵니다.

In [100]:
import os
# 노트북 파일이 있는 폴더로 이동 (예시)
os.chdir(r'C:\githome\hipython_rep')

# 변경 후 확인
print("변경 후:", os.getcwd())

변경 후: c:\githome\hipython_rep


In [101]:
X = pd.read_csv('./data1/X.csv', encoding='euc-kr')
y = pd.read_csv('./data1/y.csv', encoding='euc-kr')

<br/>

---

<br/>
<br/>

# 2. 데이터 탐색하기
* 데이터를 이해할 수 있도록 탐색과정을 수행해봅시다.


데이터의 상위 몇 개 행을 출력하여 전체 구조를 미리 확인합니다.

In [102]:
X.head()

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17
1,1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1
2,2,3197000,1639000,NaN,남성 캐주얼,관악점,2,2.000000,0.000000,1
3,3,16077620,4935000,NaN,기타,광주점,18,2.444444,0.318182,16
4,4,29050000,24000000,NaN,보석,본 점,2,1.500000,0.000000,85


In [103]:
y.head()

,cust_id,gender
0,0,0
1,1,0
2,2,1
3,3,1
4,4,0



데이터의 요약 정보나 통계 정보를 출력해 변수들의 유형과 분포를 확인합니다.

In [104]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  3500 non-null   int64  
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     1205 non-null   float64
 4   주구매상품    3500 non-null   object 
 5   주구매지점    3500 non-null   object 
 6   내점일수     3500 non-null   int64  
 7   내점당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 273.6+ KB


In [105]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   cust_id  3500 non-null   int64
 1   gender   3500 non-null   int64
dtypes: int64(2)
memory usage: 54.8 KB


In [106]:
X.describe()

,cust_id,총구매액,최대구매액,환불금액,내점일수,내점당구매건수,주말방문비율,구매주기
count,3500.000000,3.500000e+03,3.500000e+03,1.205000e+03,3500.000000,3500.000000,3500.000000,3500.000000
mean,1749.500000,9.191925e+07,1.966424e+07,2.407822e+07,19.253714,2.834963,0.307246,20.958286
std,1010.507298,1.635065e+08,3.199235e+07,4.746453e+07,27.174942,1.912368,0.289752,24.748682
min,0.000000,-5.242152e+07,-2.992000e+06,5.600000e+03,1.000000,1.000000,0.000000,0.000000
25%,874.750000,4.747050e+06,2.875000e+06,2.259000e+06,2.000000,1.666667,0.027291,4.000000
50%,1749.500000,2.822270e+07,9.837000e+06,7.392000e+06,8.000000,2.333333,0.256410,13.000000
75%,2624.250000,1.065079e+08,2.296250e+07,2.412000e+07,25.000000,3.375000,0.448980,28.000000
max,3499.000000,2.323180e+09,7.066290e+08,5.637530e+08,285.000000,22.083333,1.000000,166.000000


In [107]:
y.describe()

,cust_id,gender
count,3500.000000,3500.000000
mean,1749.500000,0.376000
std,1010.507298,0.484449
min,0.000000,0.000000
25%,874.750000,0.000000
50%,1749.500000,0.000000
75%,2624.250000,1.000000
max,3499.000000,1.000000


- cust_id: 고객번호
- gender: 여자(0), 남자(1)

<br/>

---

<br/>
<br/>

# 3. 데이터 전처리
* 전처리 과정을 통해서 머신러닝에 사용할 수 있는 형태의 데이터 준비


필요한 라이브러리를 불러옵니다.
- 인코딩 : LabelEncoder
- 데이터 표준화 : StandardScaler

In [108]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import pandas as pd

* 단순히 1부터의 숫자를 부여한 'cust_id'를 수치형 변수로 받아들이면, 결과가 왜곡될 수 있으니 컬럼을 제거합니다.

In [109]:
X = X.drop(columns=['cust_id'])

In [110]:
y = y.drop(columns=['cust_id'])

- 데이터에 결측치가 있는지 확인해보세요


In [111]:
missing_counts = X.isnull().sum()
print("결측치 개수:\n", missing_counts)

결측치 개수:
 총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64


In [112]:
missing_counts = y.isnull().sum()
print("결측치 개수:\n", missing_counts)

결측치 개수:
 gender    0
dtype: int64


- 환불금액에 2295건의 결측치 확인
- 2295건의 구매 내역은 환불을 하지 않은 것

- 결측치에 0으로 채워 넣어 모델 학습에 지장이 없도록 합니다.

In [113]:
# 결측치를 0으로 채움
X_filled = X.fillna(0)

In [114]:
missing_counts = X_filled.isnull().sum()
print("결측치 개수:\n", missing_counts)

결측치 개수:
 총구매액       0
최대구매액      0
환불금액       0
주구매상품      0
주구매지점      0
내점일수       0
내점당구매건수    0
주말방문비율     0
구매주기       0
dtype: int64


문자형 범주 데이터를 숫자로 바꾸기 위한 인코딩을 수행합니다.

In [115]:
print(X.columns.tolist())


['총구매액', '최대구매액', '환불금액', '주구매상품', '주구매지점', '내점일수', '내점당구매건수', '주말방문비율', '구매주기']


In [116]:
# 문자형 범주 컬럼 지정
label_enc_cols = ['주구매상품', '주구매지점']

# Label Encoding 적용
label_encoders = {}
for col in label_enc_cols:
    le = LabelEncoder()
    X_filled[col] = le.fit_transform(X_filled[col])
    label_encoders[col] = le  # 나중에 디코딩을 위해 저장

# 결과 확인
print(X_filled.head())

       총구매액     최대구매액       환불금액  주구매상품  주구매지점  내점일수   내점당구매건수    주말방문비율  구매주기
0  68282840  11264000  6860000.0      5      0    19  3.894737  0.527027    17
1   2136000   2136000   300000.0     21     19     2  1.500000  0.000000     1
2   3197000   1639000        0.0      6      1     2  2.000000  0.000000     1
3  16077620   4935000        0.0      5      2    18  2.444444  0.318182    16
4  29050000  24000000        0.0     15      8     2  1.500000  0.000000    85


각 데이터에 표준화를 적용하여 데이터의 스케일(크기 차이)을 맞춰줍니다.
- 평균을 0, 표준편차를 1로 맞춰서 → 데이터가 정규 분포 형태로 변환되도록 하세요

In [164]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# 가정: X_filled는 원본 DataFrame입니다. (이전에 로드되었을 것입니다.)
# 예시: X_filled = pd.read_csv('X.csv')

scaler = StandardScaler()
# 수치형 데이터 전체에 표준화 적용
X_scaled = scaler.fit_transform(X_filled)

# X_filled의 컬럼 이름을 사용하여 X_scaled를 DataFrame으로 변환
X_scaled_df = pd.DataFrame(X_scaled, columns=X_filled.columns)

<br/>

---

<br/>
<br/>

# 5-1. 모델링 - LogisticRegression

* 본격적으로 모델을 선언하고 학습시킵니다.


필요한 라이브러리를 불러옵니다.

In [118]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

모델을 선언하여 객체화시킵니다.

In [119]:
log_model = LogisticRegression(max_iter=1000, random_state=42)


모델을 학습 데이터에 맞춰 학습시킵니다.

In [166]:
X1 = X_scaled_df[['총구매액', '최대구매액', '환불금액', '내점일수', '내점당구매건수', '주말방문비율', '구매주기']]
y1 = y['gender']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
 X1, y1, test_size=0.2, random_state=42
)

<br/>

---

<br/>
<br/>

# 6-1. 예측 성능 확인해보기 - LogisticRegression

- 학습된 모델로 테스트 데이터에 대한 예측을 수행합니다.

- 학습시킨 모델의 성능을 알아봅니다
- 각 평가지표로 모델의 성능을 수치화하여 확인합니다.
- 필요한 라이브러리를 import 하고 성능을 확인해보세요 (정확도, 정밀도, 재현율, f1, confusion_matrix)

In [216]:
# GPT

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, classification_report
)

# 1. 모델 학습
log_model.fit(X_train, y_train)

# 2. 예측
y_pred = log_model.predict(X_test)

# 3. 성능 평가 (positive class = 0)
print("Accuracy:", accuracy_score(y_test, y_pred))  # Accuracy는 pos_label 설정 없음
print("Precision (pos_label=0):", precision_score(y_test, y_pred, pos_label=0))
print("Recall (pos_label=0):", recall_score(y_test, y_pred, pos_label=0))
print("F1 Score (pos_label=0):", f1_score(y_test, y_pred, pos_label=0))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))  # 여기에도 pos_label 없음
print("\nClassification Report (pos_label=0):\n", classification_report(y_test, y_pred, labels=[0,1], target_names=['Class 0 (Positive)', 'Class 1']))


Accuracy: 0.62
Precision (pos_label=0): 0.6232771822358346
Recall (pos_label=0): 0.9531615925058547
F1 Score (pos_label=0): 0.7537037037037037

Confusion Matrix:
 [[407  20]
 [246  27]]

Classification Report (pos_label=0):
                     precision    recall  f1-score   support

Class 0 (Positive)       0.62      0.95      0.75       427
           Class 1       0.57      0.10      0.17       273

          accuracy                           0.62       700
         macro avg       0.60      0.53      0.46       700
      weighted avg       0.60      0.62      0.53       700



In [168]:
def get_log_model1_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)

    print(confusion)
    print('*'*20)
    print(accuracy, precision, recall)

In [169]:
from sklearn.linear_model import LogisticRegression

log_model1= LogisticRegression(max_iter=2000)
log_model1.fit(X_train, y_train)
pred = log_model1.predict(X_test)

#정확도, 정밀도, 재현율
get_log_model1_eval(y_test, pred)

[[407  20]
 [246  27]]
********************
0.62 0.574468085106383 0.0989010989010989


In [170]:
pred_proba = log_model1.predict_proba(X_test)
pos_proba = pred_proba[:,1] #양성클래스일 확률

threshold = 0.4 #임계치
custom_proba = (pos_proba>=threshold).astype(int)  # 임계치보다 크면 1
confusion_matrix(y_test, custom_proba)
get_log_model1_eval(y_test, custom_proba)

[[277 150]
 [132 141]]
********************
0.5971428571428572 0.4845360824742268 0.5164835164835165



<br/>

---

<br/>
<br/>

# 5-2. 모델링 - DecisionTreeClassifier

* 본격적으로 모델을 선언하고 학습시킵니다.


필요한 라이브러리를 불러옵니다.

In [122]:
from sklearn.tree import DecisionTreeClassifier

모델을 선언하여 객체화시킵니다.

In [186]:
dt_model = DecisionTreeClassifier(random_state=42)

모델을 학습 데이터에 맞춰 학습시킵니다.

In [187]:
dt_model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)



<br/>
<br/>

# 6-2. 예측 성능 확인해보기 - DecisionTreeClassifier

- 학습된 모델로 테스트 데이터에 대한 예측을 수행합니다.

- 학습시킨 모델의 성능을 알아봅니다
- 각 평가지표로 모델의 성능을 수치화하여 확인합니다.
- 필요한 라이브러리를 import 하고 성능을 확인해보세요 (정확도, 정밀도, 재현율, f1, confusion_matrix)

In [219]:
y_pred = dt_model.predict(X_test)

In [218]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

tree_acc = accuracy_score(y_test, y_pred)
tree_precision = precision_score(y_test, y_pred, average='binary', pos_label=0)
tree_recall = recall_score(y_test, y_pred, average='binary', pos_label=0)
tree_f1 = f1_score(y_test, y_pred, average='binary', pos_label=0)
tree_cm = confusion_matrix(y_test, y_pred)

print(f'정확도(Accuracy): {tree_acc:.4f}')
print(f'정밀도(Precision, pos_label=0): {tree_precision:.4f}')
print(f'재현율(Recall, pos_label=0): {tree_recall:.4f}')
print(f'F1 점수(F1-score, pos_label=0): {tree_f1:.4f}')
print(f'혼동 행렬(Confusion Matrix):\n{tree_cm}')


정확도(Accuracy): 0.5286
정밀도(Precision, pos_label=0): 0.6147
재현율(Recall, pos_label=0): 0.6089
F1 점수(F1-score, pos_label=0): 0.6118
혼동 행렬(Confusion Matrix):
[[260 167]
 [163 110]]



<br/>

---

<br/>
<br/>

# 5-3. 모델링 - RandomForestClassifier

* 본격적으로 모델을 선언하고 학습시킵니다.



필요한 라이브러리를 불러옵니다.

In [128]:
from sklearn.ensemble import RandomForestClassifier

모델을 선언하여 객체화시킵니다.

In [129]:
forest_model = RandomForestClassifier(random_state=42)

모델을 학습 데이터에 맞춰 학습시킵니다.

In [130]:
forest_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [212]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=0, max_depth=8)
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_test)
accuracy_score(y_test, rf_pred)

0.6357142857142857



<br/>
<br/>

# 6-3. 예측 성능 확인해보기 - RandomForestClassifier

- 학습된 모델로 테스트 데이터에 대한 예측을 수행합니다.

- 학습시킨 모델의 성능을 알아봅니다
- 각 평가지표로 모델의 성능을 수치화하여 확인합니다.
- 필요한 라이브러리를 import 하고 성능을 확인해보세요 (정확도, 정밀도, 재현율, f1, confusion_matrix)

In [183]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [192]:
rf_pred = rf_clf.predict(X_test)

In [196]:
forest_acc = accuracy_score(y_test, y_pred)
# '여성'을 긍정 클래스로 지정
forest_precision = precision_score(y_test, y_pred, pos_label=0)
forest_recall = recall_score(y_test, y_pred, pos_label=0)
forest_f1 = f1_score(y_test, y_pred, pos_label=0)
forest_cm = confusion_matrix(y_test, y_pred)

print(f'정확도(Accuracy): {forest_acc:.4f}')
print(f'정밀도(Precision): {forest_precision:.4f}')
print(f'재현율(Recall): {forest_recall:.4f}')
print(f'F1 점수(F1-score): {forest_f1:.4f}')
print(f'혼동 행렬(Confusion Matrix):\n{forest_cm}')

정확도(Accuracy): 0.5286
정밀도(Precision): 0.6147
재현율(Recall): 0.6089
F1 점수(F1-score): 0.6118
혼동 행렬(Confusion Matrix):
[[260 167]
 [163 110]]


In [197]:
from sklearn.model_selection import GridSearchCV

params = {
    'max_depth': [8,16,24, 32],
    'min_samples_split' : [2,8,16],
    'min_samples_leaf': [1,6,12]
}

rf_clf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf, param_grid=params, cv=2, n_jobs=-1)
grid_cv.fit(X_train, y_train)

GridSearchCV(cv=2, estimator=RandomForestClassifier(n_jobs=-1, random_state=0),
             n_jobs=-1,
             param_grid={'max_depth': [8, 16, 24, 32],
                         'min_samples_leaf': [1, 6, 12],
                         'min_samples_split': [2, 8, 16]})

In [198]:
grid_cv.best_params_, grid_cv.best_score_

({'max_depth': 8, 'min_samples_leaf': 1, 'min_samples_split': 16},
 np.float64(0.6464285714285714))

In [199]:
best_model = grid_cv.best_estimator_
best_pred = best_model.predict(X_test)
accuracy_score(y_test, best_pred)

0.6328571428571429


<br/>

---

<br/>
<br/>

# 5-4. 모델링 - XGBoost

* 본격적으로 모델을 선언하고 학습시킵니다.



필요한 라이브러리를 불러옵니다.

In [200]:
import xgboost
from xgboost import XGBClassifier
print(xgboost.__version__)

1.6.2


모델을 선언하여 객체화시킵니다.

In [201]:
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss') 

모델을 학습 데이터에 맞춰 학습시킵니다.

In [202]:
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=6,
              max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)



<br/>
<br/>

# 6-4. 예측 성능 확인해보기 - XGBoost

- 학습된 모델로 테스트 데이터에 대한 예측을 수행합니다.

- 학습시킨 모델의 성능을 알아봅니다
- 각 평가지표로 모델의 성능을 수치화하여 확인합니다.
- 필요한 라이브러리를 import 하고 성능을 확인해보세요 (정확도, 정밀도, 재현율, f1, confusion_matrix)

In [203]:
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report
)

In [204]:
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

In [220]:
print(f"- 정확도 (Accuracy): {accuracy_score(y_test, y_pred):.4f}")
print(f"- 정밀도 (Precision): {precision_score(y_test, y_pred, average='binary', pos_label=0):.4f}")
print(f"- 재현율 (Recall): {recall_score(y_test, y_pred, average='binary', pos_label=0):.4f}")
print(f"- F1 점수: {f1_score(y_test, y_pred, average='binary', pos_label=0):.4f}")
print("[혼동 행렬 (Confusion Matrix)]")
print(confusion_matrix(y_test, y_pred))

- 정확도 (Accuracy): 0.5286
- 정밀도 (Precision): 0.6147
- 재현율 (Recall): 0.6089
- F1 점수: 0.6118
[혼동 행렬 (Confusion Matrix)]
[[260 167]
 [163 110]]


In [221]:
print("[분류 리포트 (Classification Report)]")
print(classification_report(y_test, y_pred))

[분류 리포트 (Classification Report)]
              precision    recall  f1-score   support

           0       0.61      0.61      0.61       427
           1       0.40      0.40      0.40       273

    accuracy                           0.53       700
   macro avg       0.51      0.51      0.51       700
weighted avg       0.53      0.53      0.53       700



<br/>

---


<br/>

## 7.  위 4가지 모델의 학습 & 예측 & 평가 결과를 확인하고 최고 성능을 내는 모델을 찾아봅시다!

- 어떤 모델이 가장 성능이 좋은가요 ?

In [222]:
# 성능 결과 저장용 리스트
results = []

# 모델 딕셔너리
models = {
    "Logistic Regression": log_model,
    "Decision Tree": dt_model,
    "Random Forest": rf_clf,
    "XGBoost": xgb_model
}

# 각 모델 성능 평가
for name, model in models.items():
    y_pred = model.predict(X_test)
    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred, average='weighted'),
        "Recall": recall_score(y_test, y_pred, average='weighted'),
        "F1 Score": f1_score(y_test, y_pred, average='weighted')
    })

# DataFrame으로 보기 좋게 출력
import pandas as pd
results_df = pd.DataFrame(results)
print(results_df.sort_values(by='F1 Score', ascending=False))


                 Model  Accuracy  Precision    Recall  F1 Score
2        Random Forest  0.635714   0.618950  0.635714  0.598113
3              XGBoost  0.607143   0.591162  0.607143  0.592859
1        Decision Tree  0.528571   0.529815  0.528571  0.529176
0  Logistic Regression  0.620000   0.604242  0.620000  0.525572


**순위**

**1. Random Forest**
- 가장 좋은 성능을 보임: 정확도(0.6357), 정밀도(0.6180), 재현율(0.6357), F1 Score(0.5981) 모두 최고 수준
- 이유: 여러 개의 의사결정 트리를 조합하여 예측을 수행하는 앙상블 모델로, 과적합을 줄이면서도 복잡한 패턴을 잘 학습
- 장점: 불균형 데이터나 다중 범주에도 강하고, 하이퍼파라미터 튜닝 없이도 기본 성능이 우수

**2. XGBoost**
- F1 점수(0.5929)는 Random Forest에 비해 약간 낮지만 여전히 우수한 성능
- 정확도(0.6071), 정밀도(0.5912), 재현율(0.6071) 모두 고른 수치를 보임
- 이유: Gradient Boosting 기반의 모델로, 오차를 반복적으로 보완하여 성능을 향상시킴
- 장점: 빠른 속도와 과적합 방지 기능, 정교한 예측

**3. Logistic Regression**
- 정확도(0.6200)는 꽤 높지만, F1 점수는 0.5256으로 정밀도와 재현율 간 균형이 떨어짐
- 이유: 선형 모델이기 때문에 복잡한 비선형 관계를 잘 포착하지 못함

**4.Decision Tree**
- 전반적인 성능이 가장 낮음 (정확도: 0.5286, F1 Score: 0.5292)
- 이유: 단일 트리는 과적합(overfitting)의 위험이 높고, 데이터의 잡음에도 민감